In [1]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [2]:
%run Util/00_imports.ipynb

# Überprüfen der $S_n$ Mengen

Bei der Bestimmung der $S_n$ Mengen wurde die Retrograde Analyse verwendet.
Um sicherzustellen, dass die Bretter die bspw. in $S_{10}$ zu finden sind, auch wirklich in <=10 Zügen beendet werden, wurden in diesem Notebook Funktionen definiert, die diesen Aspekt überprüfen

Der Vergleich der Spielbretter findet in diesem Notebook anhand der FEN-Schreibweise statt. Diese wird in zweierlei Form verwendet, in ihrer gesamten und in einer gekürzten Schreibweise. Die gekürzte Schreibweise beinhaltet nur das momentane Spielfeld, aber auch die Farbe, die gerade am Zug ist. Zur Erstellung dieser Schreibweise wurde die Funktion ``get_board_and_turn`` geschrieben. Diese erhält als Parameter eine komplette ``fen`` und gibt in der FEN-Notation das Brett und die Farbe, die am Zug ist, zurück.

In [3]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die mit Hilfe des Notebooks ``calculation.ipynb`` bestimmt werden. Diese wurden bereits in einer JSON-Datei serialisiert abgespeichert. Für dieses Notebook werden alle FENs gelesen und daraus Board-Objekte erstellt. Dies geschieht mit der Funktion ``load_s_n_sequence``. Diese erhält den Dateipfad (``filepath``) als Parameter, um aus der Datei die $S_n$ Mengen auslesen zu können.

In [4]:
def load_s_n_sequence(filepath):
    s_n_sequence_fen = []
    s_n_sequence_short_fen = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_fen = set()
        tmp_fen_short = set()
        for board in item:
            tmp_fen.add(board)
            tmp_fen_short.add(get_board_and_turn(board))
        s_n_sequence_fen.append(tmp_fen)
        s_n_sequence_short_fen.append(tmp_fen_short)
    f.close()
    return s_n_sequence_fen, s_n_sequence_short_fen

In [5]:
S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_sequence("S_n_seq_12_01.json")

FileNotFoundError: [Errno 2] No such file or directory: 'S_n_seq_12_01.json'

Die erste Funktion, die zur Überprüfung definiert wurde, lautet ``fen_in_lower_sequence``. Diese überprüft für eine mitgegebene ``fen``, ob diese in einer $S_n$ Menge liegt, für die gilt: ``n < sequence_index``. Die Funktion gibt hierfür einen booleschen Wert zurück.

In [ ]:
def fen_in_lower_sequence(fen, sequence_index, s_n_short):
    for s in s_n_short[:sequence_index]:
        if fen in s:
            return True
    return False

Als ersten Schritt wird überprüft, dass keine der Schachbretter in eine zu niedrige $S_n$ Menge eingeordnet wurde. Dies wird mit der Funktion ``every_move_of_sequence_in_lower`` umgesetzt. Hierfür erhält sie eine ``sequence_index``, die überprüft werden soll. Für jede Spielsiuation wird folgendes überprüft:

$\forall board \in S_n : move \in board.legal\_moves \implies board.push(move) \in S_m, m<n$

Als Ergebnis gibt die Funktion eine boolesche Variable zurück, die ``True`` zurückgibt, falls die Bedingung erfüllt ist, ansonsten ``False``.

In [ ]:
def every_move_of_sequence_in_lower(sequence_index, s_n_fen, s_n_short):
    print("Checking S_" + str(sequence_index) + "...")
    sequence_fen = s_n_fen[sequence_index]
    for fen in sequence_fen:
        cur_board = chess.Board(fen)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if not fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    print("Fen: " + cur_short + " not in lower S")
                    return False
                cur_board.pop()
            return True
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    in_lower = True
                cur_board.pop()
            return in_lower


In [ ]:
for i in range(len(S_N_Sequence_fen)):
    every_move_of_sequence_in_lower(i, S_N_Sequence_fen, S_N_Sequence_fen_short)

Für die Bestimmung der Züge, die für eine Spielsituation benötigt werden, bis es beendet ist, müssen jegliche Spielzüge durchgegangen werden. Zur Berechnung wurde dazu die Funktion ``find_fen_in_sequence`` definiert, die in einer mitgegeben Liste von gekürzten $S_n$ Mengen (``s_n_short``) den Mengenindex zurück gibt, in der die gekürzte ``fen`` gefunden wurde.

In [ ]:
def find_fen_in_sequence(fen, s_n_short):
    short = get_board_and_turn(fen)
    for i in range(len(s_n_short)):
        if short in s_n_short[i]:
            return i

Die Funktion ``find_move_count`` kann nun für folgende Überprüfung verwendet werden:

$ \forall board \in S_n \implies max(find\_move\_count(board)) = n $

Hierbei berechnet ``find_move_count()`` jeden Ablauf einer Schach-Partie bis ein Schachmatt erzielt worden ist. Da aber die Spielsituation in die Menge $S_n$ zugeordnet wurde, muss auf jeden Fall eine dieser Zahlen den Wert n betragen, da ansonsten das Schachbrett der falschen $S_n$ Menge zugeordnet wurde.

Die Implementierung der Funktion ``find_move_count`` erfolgt rekursiv. Die Funktion ruft sich nämlich solange selbst auf, bis ein Schachmatt erzielt worden ist. Für einen derartigen Aufruf wird die ``sequence_index``, in der sich die ``fen`` befindet, mitgegeben. Außerdem muss das ``move_set``, das auch als Rückgabewert fungiert, mitgegeben werden, da darin die Anzahl der Züge gespeichert werden, die zum Lösen jeglicher Spielabläufe benötigt werden. Der Parameter ``move_count`` resultiert ebenfalls durch die Rekursion, da in ihr die aktuelle Anzahl an ausgeführten Spielzügen gespeichert wird. Aufgrund des Aufrufs der Funktion ``find_fen_in_sequence`` wird zusätzlich noch die Liste der gekürzten $S_n$ Mengen benötigt (``s_n_short``). Die Funktion gibt letztendlich eine Menge an Spielzügen zurück, die für die unterschiedlichen Abläufe der Spielsituation benötigt wurden.

In [ ]:
def find_move_count(sequence_index, fen, move_count, move_set, s_n_short):
    cur_board = chess.Board(fen)
    moves = cur_board.legal_moves
    if moves.count() == 0 and sequence_index == 0:
        move_set.add(move_count)
        return move_set
    if not cur_board.turn:
        # print("Black:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            move_count += 1
            cur_fen = cur_board.fen()
            new_seq = find_fen_in_sequence(cur_fen, s_n_short)
            # print(new_seq, ";" , cur_fen)
            move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
            move_count -= 1
            cur_board.pop()
    else: 
        # print("White:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            cur_fen = get_board_and_turn(cur_board.fen())
            if fen_in_lower_sequence(cur_fen, sequence_index, s_n_short):
                new_seq = find_fen_in_sequence(cur_fen, s_n_short)
                move_count += 1
                move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
                move_count -= 1
            cur_board.pop()
    return move_set

In [ ]:
find_move_count(20, "8/8/8/3K4/8/5R2/2k5/8 b - - 0 1", 0, set(), S_N_Sequence_fen_short)

Zur Überprüfung, dass nun kein Board der falschen $S_n$ Menge zugeordnet wurde, wurde die Funktion ``check_boards_in_correct_sequence`` definiert. Diese überprüft für einen ``sequence_index`` in einer mitgegebenen Liste von $S_n$ Mengen (``s_n_fen``), ob die Spielsituationen der Menge an dem Index ``sequence_index`` auch maximal $n$ Züge bis zum Ende der Partie benötigen. Dies gelingt mit der Funktion ``find_move_count``, die zusätzlich noch die gekürzte Schreibweise der $S_n$ Mengen benötigt (``s_n_short``). Die Funktion gibt in Form einer Konsolenausgabe an, welche der FENs in eine falsche $S_n$ Menge zugeordnet worden sind.

In [ ]:
def check_boards_in_correct_sequence(sequence_index, s_n_fen, s_n_short):
    invalid = 0
    print("Comparing S_" + str(sequence_index) + "...")
    for fen in s_n_fen[sequence_index]:
        move_set = find_move_count(sequence_index, fen, 0, set(), s_n_short)
        if max(move_set) != sequence_index:
            invalid += 1
            print(fen)
    print("Invalid boards: " + str(invalid))

In [ ]:
check_boards_in_correct_sequence(4, S_N_Sequence_fen, S_N_Sequence_fen_short)